# Imputación de Datos Faltantes usando KNN
Este notebook realiza la imputación de datos faltantes en el **dataset de test** utilizando el algoritmo de k-vecinos más próximos (KNN).  
Este proceso se realiza después de la normalización para asegurar que todas las variables estén en la misma escala.

## Importación de librerías y carga de datos

In [1]:
import pandas as pd
import numpy as np
from tabulate import tabulate

# Cargar el dataset de test normalizado
df_test = pd.read_csv('../datasets/normal_test_pre_imputation.csv', index_col=0)

## Preparación para la imputación KNN

### 1. Separación de datos completos e incompletos

In [2]:
# Identificar filas completas (sin valores faltantes)
complete_rows = df_test.dropna()

# Identificar filas con valores faltantes
incomplete_rows = df_test[df_test.isnull().any(axis=1)]

print("Número de filas completas:", len(complete_rows))
print("Número de filas incompletas:", len(incomplete_rows))

Número de filas completas: 378
Número de filas incompletas: 22


### 2. Función para calcular distancia euclidiana

In [3]:
def euclidean_distance(row1, row2, columns):
    """
    Calcula la distancia euclidiana entre dos filas usando solo las columnas especificadas
    
    Parámetros:
    row1, row2 (Series): Filas del DataFrame a comparar
    columns (list): Lista de columnas a usar para el cálculo
    
    Retorna:
    float: Distancia euclidiana entre las filas
    """
    return np.sqrt(sum((row1[columns] - row2[columns])**2))

### 3. Función para encontrar k vecinos más cercanos

In [4]:
def get_k_neighbors(target_row, complete_data, columns_to_use, k=5):
    """
    Encuentra los k vecinos más cercanos para una fila dada
    
    Parámetros:
    target_row (Series): Fila para la cual buscar vecinos
    complete_data (DataFrame): Dataset con filas completas
    columns_to_use (list): Columnas a usar para calcular distancias
    k (int): Número de vecinos a encontrar
    
    Retorna:
    list: Índices de los k vecinos más cercanos
    """
    distances = []
    for idx, row in complete_data.iterrows():
        dist = euclidean_distance(target_row, row, columns_to_use)
        distances.append((idx, dist))

    # Ordenar por distancia y obtener los k más cercanos
    distances.sort(key=lambda x: x[1])
    return [idx for idx, _ in distances[:k]]

### 4. Función de imputación

In [5]:
def impute_missing_values(df, k=5):
    """
    Imputa valores faltantes usando KNN
    
    Parámetros:
    df (DataFrame): Dataset con valores faltantes
    k (int): Número de vecinos a considerar
    
    Retorna:
    DataFrame: Dataset con valores imputados
    """
    df_imputed = df.copy()
    complete_data = df.dropna()

    # Para cada fila con valores faltantes
    for idx, row in df[df.isnull().any(axis=1)].iterrows():
        # Identificar columnas con valores faltantes en esta fila
        missing_cols = row[row.isnull()].index

        # Identificar columnas disponibles para calcular distancias
        available_cols = row[row.notnull()].index

        # Encontrar k vecinos más cercanos
        neighbors_idx = get_k_neighbors(row, complete_data, available_cols, k)

        # Imputar cada columna faltante con la media de los vecinos
        for col in missing_cols:
            neighbor_values = complete_data.loc[neighbors_idx, col]
            df_imputed.loc[idx, col] = neighbor_values.mean()

    return df_imputed

## Aplicación de la imputación KNN

In [6]:
# Aplicar la imputación
df_test_imputed = impute_missing_values(df_test, k=5)

# Verificar que no quedan valores faltantes
print("\nVerificación de valores faltantes después de imputación:")
print(df_test_imputed.isnull().sum())

# Verificar las dimensiones finales
print("\nDimensiones del dataset después de imputación:", df_test_imputed.shape)


Verificación de valores faltantes después de imputación:
Defense Against the Dark Arts    0
Herbology                        0
Potions                          0
Charms                           0
Flying                           0
Best Hand                        0
Age                              0
dtype: int64

Dimensiones del dataset después de imputación: (400, 7)


### Guardado del dataset final
Grabación del Dataset de 'test' normalizado y sin datos faltantes.

In [7]:
# Guardar el dataset completo normalizado e imputado
df_test_imputed.to_csv('../datasets/normal_test.csv')

print("\nColumnas en el dataset final:\n\t", df_test_imputed.columns.tolist())
print("\nPrimeras filas del dataset final:\n")
print(tabulate(df_test_imputed.head(), headers='keys', tablefmt='fancy_grid', floatfmt='.3f', showindex=True))


Columnas en el dataset final:
	 ['Defense Against the Dark Arts', 'Herbology', 'Potions', 'Charms', 'Flying', 'Best Hand', 'Age']

Primeras filas del dataset final:

╒═════════╤═════════════════════════════════╤═════════════╤═══════════╤══════════╤══════════╤═════════════╤════════╕
│   Index │   Defense Against the Dark Arts │   Herbology │   Potions │   Charms │   Flying │   Best Hand │    Age │
╞═════════╪═════════════════════════════════╪═════════════╪═══════════╪══════════╪══════════╪═════════════╪════════╡
│       0 │                          -1.264 │       0.361 │    -0.737 │   -0.129 │   -0.363 │       1.000 │ -1.710 │
├─────────┼─────────────────────────────────┼─────────────┼───────────┼──────────┼──────────┼─────────────┼────────┤
│       1 │                           0.783 │       0.351 │     0.812 │    1.370 │   -0.493 │       0.000 │  0.904 │
├─────────┼─────────────────────────────────┼─────────────┼───────────┼──────────┼──────────┼─────────────┼────────┤
│       2 │   

## Nota
- El dataset final `normal_test.csv` contiene los 400 registros completos, normalizados y sin valores faltantes.
- Este dataset está listo para ser usado en el proceso de clasificación.
- Las variables numéricas están normalizadas usando los mismos parámetros (media y desviación estándar) que se usaron en el dataset de entrenamiento.